#### Testing parallelized combination indices generation with pycuda

This indices will form the inner loop in combinations 

In [49]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [50]:
mod = SourceModule('''
__global__ void comb_indices(int* left,int* right, int* start, int* stop)
{
    int len = stop[0]-start[0];
    int i = threadIdx.x + blockDim.x*blockIdx.x;
    int j = threadIdx.y + blockDim.y*blockIdx.y;
    if ( i<len && j<=i) 
    {
        left[len*(len+1)/2-(i*(i+1)/2)-j-1] = len-1-i;
        right[len*(len+1)/2 - (i*(i+1)/2) -j -1] = len-1-j;
    }
}
''')
func = mod.get_function("comb_indices")

In [51]:
start = numpy.array([0]).astype(numpy.int32)
stop = numpy.array([8]).astype(numpy.int32)
length = (stop-start)*(stop-start+1)/2
length = length.astype(numpy.int32)
left = numpy.empty(length).astype(numpy.int32)
right = numpy.empty(length).astype(numpy.int32)

In [52]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start),cuda.In(stop), 
     block=(16,16,1), grid=(1,1))

In [53]:
print(right)

[0 1 2 3 4 5 6 7 1 2 3 4 5 6 7 2 3 4 5 6 7 3 4 5 6 7 4 5 6 7 5 6 7 6 7 7]


In [54]:
print(left)

[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 4 4 4 4 5 5 5 6 6 7]
